In [2]:
#%pip install --upgrade --quiet  langchain langchain-openai
#%pip install pypdf
#%pip install chromadb

In [3]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

In [4]:
# Import necessary libraries
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import TokenTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import ConversationalRetrievalChain
from langchain_community.chat_models import ChatOpenAI


#File paths for the PDF documents
pdf_paths = [
    "/home/habte/Downloads/Raptor Contract.docx.pdf",
    "/home/habte/Downloads/Raptor Q&A2.docx.pdf",
    "/home/habte/Downloads/Robinson Advisory.docx.pdf",
    "/home/habte/Downloads/Robinson Q&A.docx.pdf"
    ]

# Load PDF and split into tokens for all documents
pdf_data = []
for path in pdf_paths:
    loader = PyPDFLoader(path)
    pdf_data.extend(loader.load())

text_splitter = TokenTextSplitter(chunk_size=1000, chunk_overlap=0)
split_data = text_splitter.split_documents(pdf_data)

# Set up ChromaDB with vector embeddings
collection_name = "contracts_collection"
local_directory = "contracts_vect_embedding"
persist_directory = os.path.join(os.getcwd(), local_directory)

openai_key = os.environ.get('OPENAI_API_KEY')  # Use a proper environment variable name
embeddings = OpenAIEmbeddings(openai_api_key=openai_key)
vect_db = Chroma.from_documents(
    split_data,
    embeddings,
    collection_name=collection_name,
    persist_directory=persist_directory
)
vect_db.persist()

# Initialize the OpenAI language model
chat_model = ChatOpenAI(openai_api_key=openai_key, model_name="gpt-3.5-turbo")
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
chat_qa = ConversationalRetrievalChain.from_llm(
    chat_model,
    vect_db.as_retriever(),
    memory=memory
)


/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(
/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [5]:
# Chat loop
chat_history = []

while True:
    query = input('You: ')  # Prompt the user to input a question

    if query.lower() == 'done':  # Check if the user wants to end the conversation
        break

    response = chat_qa({"question": query, "chat_history": chat_history})  # Retrieve response

    chat_history.append({"role": "user", "content": query})  # Update chat history
    chat_history.append({"role": "assistant", "content": response["answer"]})

    print('Assistant:', response["answer"])  # Print the assistant's response

/home/habte/anaconda3/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Assistant: Hello! How can I assist you today?
Assistant: I don't have specific information about contracts in general. If you have a specific question about a contract or a particular type of contract, feel free to ask!
Assistant: The provided context outlines various types of contractual obligations that the Company must disclose to the Buyer in specific schedules. These obligations include employment or consultancy agreements, distribution contracts, powers of attorney, lease agreements, loans to affiliates, and agreements with related parties. The Company must ensure that these contracts are enforceable, not in breach, and will continue to be enforceable after the transaction. The Company is also required to disclose any related party transactions and ensure that employees have the necessary permits and classifications. Additionally, the Company should not have any outstanding payments or disputes with current or former employees, directors, or other service providers.
Assistant: Co